In [1]:
import pandas as pd
import pandas as pd
import clickhouse_driver as ch
from hw import HoltWinters

df = pd.read_csv('test.csv').drop('Unnamed: 0', axis=1)

In [1]:
import pandas as pd
import clickhouse_driver as ch
from hw import HoltWinters

with open('q2.sql', 'r') as f:
    q = f.read()

sqlCH = ch.Client(host="192.168.235.72",
                    user="valferov",
                    password="bUfMupl1kDgK1337")

df = pd.DataFrame(sqlCH.execute(q))
df.to_csv('test.csv')

In [2]:
df.columns = ['datetime', 'campaings', 'shows']

campaigns = df['campaings'].drop_duplicates()
test = df.loc[df['campaings'] == campaigns[0]]
test = test.sort_values('datetime')
test = test.drop(test.loc[test['shows'] == 0].index)\
            .drop(test.index.values[-5:])\
            .reset_index().drop('index', axis=1)
test['datetime'] = pd.to_datetime(test['datetime'])
test

,datetime,campaings,shows
0,2022-07-19 09:00:00,RU | Гидрактив,374665
1,2022-07-19 10:00:00,RU | Гидрактив,398972
2,2022-07-19 11:00:00,RU | Гидрактив,359010
3,2022-07-19 12:00:00,RU | Гидрактив,335606
4,2022-07-19 13:00:00,RU | Гидрактив,324781
...,...,...,...
188,2022-07-27 05:00:00,RU | Гидрактив,371695
189,2022-07-27 06:00:00,RU | Гидрактив,475746
190,2022-07-27 07:00:00,RU | Гидрактив,562392
191,2022-07-27 08:00:00,RU | Гидрактив,616918


In [3]:
from scipy.optimize import minimize
from cross_val import CVScore

x=[0,0,0]

timeseriesCV = CVScore(test['shows'])

opt = minimize(timeseriesCV.timeseriesCVscore, x0=x, method='TNC', 
                bounds=((0, 1), (0, 1), (0, 1)), options={'maxiter': 10000})

opt

     fun: 11711655513.148415
     jac: array([-5.72204587e+02,  3.66617789e+11, -3.45809935e+07])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 236
     nit: 11
  status: 1
 success: True
       x: array([0.91976281, 0.        , 1.        ])

# Build model & validation

In [4]:
hw = HoltWinters(test['shows'], slen=24, alpha=opt.x[0], 
                    beta=opt.x[1], gamma=opt.x[2], n_preds=0,
                    scaling_factor=2.56)
hw.triple_exponential_smoothing()

validation = pd.DataFrame(dict(shows=test['shows'],
                                datetime=test['datetime'],
                                predict=hw.result))
validation

validation['SSR'] = (validation['shows'] - validation['predict'])**2
validation['SST'] = (validation['shows'] - validation['shows'].mean())**2
check = validation['SSR'].sum()/validation['SST'].sum()
check

3.2742465495719748e-06

# Forecast

In [5]:
hw_forecast = HoltWinters(test['shows'], slen=24, alpha=opt.x[0], 
                    beta=opt.x[1], gamma=opt.x[2], n_preds=10,
                    scaling_factor=2.56)
hw_forecast.triple_exponential_smoothing()

forecast_arr = []
for i in range(int(len(test)/3)):
    forecast_arr.append(test['datetime'].values[-1]+pd.Timedelta(hours=i))
forecast = pd.DataFrame(forecast_arr, columns=['datetime'])
forecast['forecast'] = hw_forecast.result[-len(forecast['datetime']):]

In [27]:
pd.concat([test[['datetime', 'shows']], forecast[['datetime', 'forecast']]])

,datetime,shows,forecast
0,2022-07-19 09:00:00,374665.0,NaN
1,2022-07-19 10:00:00,398972.0,NaN
2,2022-07-19 11:00:00,359010.0,NaN
3,2022-07-19 12:00:00,335606.0,NaN
4,2022-07-19 13:00:00,324781.0,NaN
...,...,...,...
59,2022-07-29 20:00:00,NaN,512804.119229
60,2022-07-29 21:00:00,NaN,482743.116114
61,2022-07-29 22:00:00,NaN,400906.908393
62,2022-07-29 23:00:00,NaN,346289.967406


# Build plot

In [6]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=test['datetime'].values, 
                        y=test['shows'].values,
                        name=f'campaign {campaigns[0]}'))
fig.add_trace(go.Scatter(x=test['datetime'].values,
                            y=hw.result, name='predict'))
fig.add_trace(go.Scatter(x=forecast['datetime'].values, 
                            y=forecast['forecast'].values,
                            name=f'forecast for {50} values'))
fig.update_layout(
    font_family="Courier New",
    title_text=f'Forecast for campaign {campaigns[0]} with accurancy {round(100-check, 10)}%'
)
fig.show()

In [36]:
forecast

,datetime,forecast
0,2022-07-27 09:00:00,558199.499467
1,2022-07-27 10:00:00,568726.627182
2,2022-07-27 11:00:00,569447.553802
3,2022-07-27 12:00:00,570138.718981
4,2022-07-27 13:00:00,589215.546619
...,...,...
59,2022-07-29 20:00:00,512804.119229
60,2022-07-29 21:00:00,482743.116114
61,2022-07-29 22:00:00,400906.908393
62,2022-07-29 23:00:00,346289.967406


In [69]:
import plotly.graph_objects as go

forecastPeriod=10

forecast = exp.forecast(forecastPeriod)

fig = go.Figure()

fig.add_trace(go.Scatter(x=test['datetime'].values, 
                        y=test['shows'].values,
                        name=f'Campaigns {campaigns[0]}'))

fig.add_trace(go.Scatter(x=[test['datetime'].values[-1] + pd.Timedelta(hours=i) 
                                for i in range(forecastPeriod)],
                            y=forecast, 
                            name=f'Forecast for {forecastPeriod} hours'))

fig.show()